In [2]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time
import csv

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata
from torchaudio import transforms

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

In [3]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data/data") #need to change this back before pushing ***
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []
for data in wav_files: #in the my_datasets code
    data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])
    label_array.append(data[1])

max_len = max([len(data) for data in data_array])
data_array = [np.resize(data, 48000*5) for data in data_array]
print(len(data_array[0]))

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_25742/3951241112.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])


240000


In [4]:
# split into training and testing
train_data, test_data, train_labels, test_labels = train_test_split(data_array, label_array, train_size=0.5, test_size=0.5)
#print(data.shape, train_data.shape, test_data.shape)

train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)
train_y_ten, test_y_ten = torch.tensor(train_labels), torch.tensor(test_labels)

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_25742/2606776394.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)


In [5]:
#implementing the Mel-frequency cepstrum coefficients as feature vectors
mfcc = transforms.MFCC(sample_rate=48000, n_mfcc=40)
#print(train_ten.shape)
#mel_spectrogram = transforms.MelSpectrogram(sample_rate=48000, n_fft=400, n_mels=1)
#print("2", mel_spectrogram(train_ten[0].float()).shape)
#print("2.5", mfcc(mel_spectrogram(train_ten[0].float())).shape)
#print(mfcc(train_ten[0].float()).shape)
#mfcc_features = np.zeros((562, 48040))
#for row in range(562):
#    mel_coef = np.array(mfcc(train_ten[row].float()))
#    mel_coef = mel_coef.reshape((1, 48040))
#    mfcc_features[row] = mel_coef

/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [8]:
#implementing the Mel-frequency cepstrum coefficients as feature vectors
mfcc2 = transforms.MFCC(sample_rate=48000, n_mfcc=40)
#mel_spectrogram = transforms.MelSpectrogram(sample_rate=48000, n_fft=400, n_mels=1)
#print("2", mel_spectrogram(train_ten[0].float()).shape)
#print("2.5", mfcc(mel_spectrogram(train_ten[0].float())).shape)
#mfcc_test_features = np.zeros((562, 48040))
#for row in range(562):
#    mel_coef = np.array(mfcc2(test_ten[row].float()))
#    mel_coef = mel_coef.reshape((1, 48040))
#    mfcc_test_features[row] = mel_coef

In [17]:
# convolution layer
# structure comes from IBM website
# will use mel_coef as feautre vectors

#mfcc_features = torch.tensor(mfcc_features)

CONV_KERNEL = 3
conv_layer = torch.nn.Conv1d(40, 50, CONV_KERNEL)

#conv_out = conv_layer(mfcc_features)

#print(conv_out.shape)



In [7]:
# pooling layer
KERNEL_SIZE = 3
pooling_layer = torch.nn.MaxPool1d(KERNEL_SIZE)

#pooled_out = pooling_layer(conv_out)

#pooled_out = pooled_out.flatten()
#pooled_out.shape

In [38]:
# fully connected layer
#implementing kernel SVM
class MLP(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(in_features=19950, out_features=1000)
    self.fc2 = torch.nn.Linear(in_features=1000, out_features=100)
    self.fc3 = torch.nn.Linear(in_features=100, out_features=10)
    self.fc4 = torch.nn.Linear(in_features=10, out_features=1)

  def forward(self, x):
    x1 = torch.relu(self.fc1(x))
    x2 = torch.relu(self.fc2(x1))
    x3 = torch.relu(self.fc3(x2))
    x4 = self.fc4(x3) #usually left linear
    return torch.softmax(x4, dim=0) #for classification

In [20]:
train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)
train_y_ten, test_y_ten = torch.tensor(train_labels), torch.tensor(test_labels)

train_dataset = tdata.TensorDataset(train_ten, train_y_ten)

train_loader = tdata.DataLoader(train_dataset, batch_size=1) #this is the data loader (just doing 1 point at a time)

In [47]:
n_epochs = 10 #essentially number of iterations

#instantiating a model
model = MLP()

#loss function
criterion = torch.nn.CrossEntropyLoss() #binary cross-entropy loss, clamps log values to protect against extremes

#optimizer mechanism
optimizer = torch.optim.Adam(lr=1e-2, params=model.parameters())

loss_array = np.zeros(n_epochs)
#something about test accuracy here - see final code
for epoch_idx in range(n_epochs):
  for batch in train_loader:
    data_batch, labels_batch = batch

    #find mfcc of audio file
    mfcc_data = mfcc(data_batch.float())

    # convolutional layer
    conv_out = conv_layer(mfcc_data)

    # pooling layer
    pooled_out = pooling_layer(conv_out)
    pooled_out = pooled_out.flatten()
    
    # move to GPU if available - data_batch = data_batch.cuda() etc.

    #predict
    labels_pred = model(pooled_out.to(torch.float32))

    #compute loss
    loss = criterion(labels_pred.flatten(), labels_batch.to(torch.float32)) #note: cross entropy is not symmetric

    loss_array[epoch_idx] += loss.item()/data_batch.shape[0] #mean of loss
    #plotting the precision
    #with torch.no_grad():
      #test_pred = model(test_ten).flatten()
      #test_loss =
 
    #backprop
    optimizer.zero_grad()
    loss.backward() #backprop from pytorch
    optimizer.step() #all params optimized

  #if epoch_idx%50 == 0:
  #  fig.data[0].y = loss_array[:epoch_idx]
  #  fig.update_yaxes(type='log')

In [18]:

mfcc_data = mfcc(torch.tensor(train_data[0]).float())

# convolutional layer
conv_out = conv_layer(mfcc_data)

# pooling layer
pooled_out = pooling_layer(conv_out)
pooled_out = pooled_out.flatten()
print(pooled_out.shape)

torch.Size([19950])


In [49]:
test_pred = []
with torch.no_grad():
  for test in test_ten:
    test = mfcc_data = mfcc(test.float())

    # convolutional layer
    conv_test = conv_layer(test)

    # pooling layer
    pooled_test = pooling_layer(conv_test)
    pooled_test = pooled_test.flatten()
    test_pred.append(model(pooled_test.to(torch.float32)).flatten())
test_pred = torch.tensor(test_pred)
print(test_pred)
print(classification_report(test_y_ten, test_pred))

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`